In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import rc
import math
import pysam
from scipy.stats import binom
from multiprocessing import Pool
import pybedtools
import os
from collections import defaultdict
from functools import partial

In [2]:
pd.set_option('display.max_columns', 500)

In [3]:
input_dir = '/home/pjaganna1/projects/STAMP/isoform_pipeline/RPS2_ADAR_test'
output_dir = '/home/pjaganna1/projects/STAMP/isoform_pipeline/RPS2_ADAR_test'

In [4]:
#sites= pd.read_csv('/projects/ps-yeolab5/d2lorenz/Nanopore/STAMP_Paper_2020_07_01/hg19/beds/RBFOX_0-based.0.99sailor.fraction.APO.rmSNP.bedgraph',sep='\t', header=None)

In [5]:
#sites

In [6]:
#len(sites)

In [7]:
# samfile = pysam.AlignmentFile(os.path.join(input_dir, "RPS2_PacBio_181120_merged.sorted.bam"), "rb")
samfile = pysam.AlignmentFile('/projects/ps-yeolab3/ekofman/sc_STAMP_pipeline/STAMP/workflow_peakcaller/small_example_materials/peakcalling_outputs/window.bam', 'rb')

In [29]:
def build_fasta_dict(fasta):
    """returns dictionary of fasta id:sequence"""
    fasta_dict = {}
    FastaFile = open(fasta, 'rU')
    for rec in SeqIO.parse(FastaFile, 'fasta'):
        name = rec.id
        seq = str(rec.seq)
        fasta_dict[name] = seq
    FastaFile.close()
    return fasta_dict


def get_ct_edit_fraction(row, strand):
    """
    Calculates the C>T (or G>A) edit fraction at a given position.
    """
    if strand == '+':
        if row['ref'] == 'A' and row['A'] > 0:
            return row['G']/(row['A']+row['C']+row['G']+row['T']) # not sure if calculating edit fraction 
    elif strand == '-':
        if row['ref'] == 'T' and row['T'] > 0:
            return row['C']/(row['A']+row['C']+row['G']+row['T']) # not sure if calculating edit fraction 
    return -1


def get_position_matrix(bam, chrom, start, end, strand, reffile, stepper='nofilter', debug_read="", min_base_quality=0):
    reference = pybedtools.BedTool.seq([chrom,0,end],reffile)
    infile = pysam.AlignmentFile(bam, "rb", reference_filename=reffile)
    count = start # running counter for each position added
    alphabet = {}
    edits = defaultdict(dict)
    positions = []
    offset = 0
    max_offset = 0
    MAX_DEPTH = 10000000
    for pileupcolumn in infile.pileup(chrom, start, end, stepper=stepper, max_depth=MAX_DEPTH, min_base_quality=min_base_quality):
        if pileupcolumn.pos >= start:  # pileupcolumn.pos is the lowest genomic coordinate (0-based) across all reads at <start> 
            st = ""  # Store all bases foreach read at ONE position (count)
            if count >= (end) or pileupcolumn.pos >= end:  # I think this works because of sorted reads?
                break
            """
            if there is no read coverage at the beginning positions
            """
            while(count < pileupcolumn.pos):
                alphabet['A'] = 0
                alphabet['T'] = 0
                alphabet['C'] = 0
                alphabet['G'] = 0
                alphabet['del'] = 0
                alphabet['ins'] = 0
                alphabet['ref'] = reference[pileupcolumn.reference_pos].upper()
                positions.append(alphabet)
                alphabet = {}
                count = count + 1
            """
            count all the bases at each position, store in string (st)
            """
            for pileupread in pileupcolumn.pileups: # for each pileup read

                edits[pileupread.alignment.query_name][count] = 0
                if (not pileupread.is_del) and (not pileupread.is_refskip):
                    """
                    THIS BLOCK DETERMINES IF C>T or G>A EDIT EXISTS
                    """
                    if strand == '+' and reference[count] == 'C' and pileupread.alignment.query_sequence[pileupread.query_position] == 'T':
                        edits[pileupread.alignment.query_name][count] = 1
                    elif strand == '-' and reference[count] == 'G' and pileupread.alignment.query_sequence[pileupread.query_position] == 'A':
                        edits[pileupread.alignment.query_name][count] = 1
                    st = st + pileupread.alignment.query_sequence[pileupread.query_position]
                    
                    if(pileupread.alignment.query_name==debug_read):
                        print("st at refpos: {} and querypos: {}: {}".format(count, pileupread.query_position, st))
                elif pileupread.is_del:
                    st = st + 'd'
                    if(pileupread.alignment.query_name==debug_read):
                        print("st at refpos: {} and querypos: {}: {}".format(count, pileupread.query_position, st))
                elif pileupread.is_refskip:
                    st = st + 'i'
                    if(pileupread.alignment.query_name==debug_read):
                        print("st at refpos: {} and querypos: {}: {}".format(count, pileupread.query_position, st))
                else:
                    print("THIS READ", pileupread.alignment.query_name, count, pileupcolumn.reference_pos, pileupread.query_position)
                    print( pileupread.is_del, pileupread.is_refskip, pileupread.indel)
                    st = st + '-'
                    if(pileupread.alignment.query_name==debug_read):
                        print("st at refpos: {} and querypos: {}: {}, [{}]".format(count, pileupread.query_position, st, pileupread.alignment.query_sequence[76]))
            """
            count number of occurrences of each base
            """
            alphabet['A'] = st.count('A')
            alphabet['T'] = st.count('T')
            alphabet['C'] = st.count('C')
            alphabet['G'] = st.count('G')
            alphabet['del'] = st.count('d')
            alphabet['ins'] = st.count('i')
            alphabet['ref'] = reference[pileupcolumn.reference_pos].upper()
            count = count + 1
            # print(alphabet)
            
            positions.append(alphabet)
            alphabet = {}
    """
    If there are positions in the end without read coverage
    """
    while(count < end):
        # count = count + 1
        alphabet['A'] = 0
        alphabet['T'] = 0
        alphabet['C'] = 0
        alphabet['G'] = 0
        alphabet['del'] = 0
        alphabet['ins'] = 0
        alphabet['ref'] = reference[count].upper()
        
        for read_name in edits.keys():
            edits[read_name][count] = np.nan
            
        count = count + 1
        positions.append(alphabet)
        
    """
    c>t edit calculations. likely doesn't take too much time/effort
    """
    df = pd.DataFrame(positions)
    df.index = df.index + start  # re-align index to match genomic coordinates.
    df['edit_fraction'] = df.apply(partial(get_ct_edit_fraction, strand=strand), axis=1)
    return df, edits

# Save position info into a pandas DataFrame, and read info into a dictionary
- "debug read" just spits out extra information about a single read, good if there are bugs and want to isolate

In [30]:
reffile = '/home/pjaganna1/projects/genomes/hg19/annotations/hg19.fa'

df, edits = get_position_matrix(
    bam="/projects/ps-yeolab3/ekofman/sc_STAMP_pipeline/STAMP/workflow_peakcaller/small_example_materials/peakcalling_outputs/window.bam",
    chrom='chr1', 
    start=567380, 
    end=567417, 
    strand='+',
    reffile=reffile, 
    stepper='nofilter',
    debug_read=''
)

In [31]:
df

,A,T,C,G,del,ins,ref,edit_fraction
567380,2,0,10168,0,0,0,C,-1.000000
567381,10302,0,1,0,0,0,A,0.000000
567382,0,10462,1,0,0,0,T,-1.000000
567383,10453,2,2,0,0,0,A,0.000000
567384,10459,0,8,2,0,0,A,0.000191
567385,0,10461,1,1,0,0,T,-1.000000
567386,0,5,10444,0,0,0,C,-1.000000
567387,10282,3,1,0,0,0,A,0.000000
567388,1,10252,0,0,0,0,T,-1.000000
567389,2,2,10265,0,0,0,C,-1.000000


# Modification to save read info:
- "edits" contains the dictionary whose keys are read names that cover a given genomic position.
- NaNs represent genomic coordinates not covered by the read
- "0" indicates "no edit" while "1" indicates either a C>T or G>A edit
- Below is some code to parse out, for a given read (probably can parse out all at once) return all positions with an edit. Indices (positions) are 0-based, so it should match (IGV coordinates - 1), which are 1-based.

In [32]:
for readname, editpositions in edits.items():
    for editposition in editpositions.keys():
        if editposition == 567386 and editpositions[editposition] == 1:
            print(readname)

A00475:396:HW7FGDRXY:1:2151:25798:36182
A00475:396:HW7FGDRXY:1:1214:28980:31516
A00475:396:HW7FGDRXY:1:1214:28989:31657
A00475:396:HW7FGDRXY:1:1264:12066:14246
A00475:396:HW7FGDRXY:1:1253:11568:18082


In [33]:
edits_df = pd.DataFrame()
l =[]

for k,v in edits.items():
    for kv,vv in v.items():
        if vv>0:
            print(k, kv)

In [ ]:
x[x['10edac1d-1630-4670-8da9-d9ca3763cb78']>0]